### Goal
Run flows using sdk and allowing for external configuration of RAG parameters. 

### Check Connections

In [ ]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

### Test the flow

In [ ]:
output = pf.flows.test(
    "../../rag-cosmos-nosql/flow.dag.yaml",
    inputs={
        "chat_history": [],
        "query": "What is the growth rate of Azure ML revenue in FY23Q1?",
    },
)

print(output)

### Run the flow with the benchmark data file

In [ ]:
flow_path = "../../rag-cosmos-nosql/flow.dag.yaml"
data_path = "../../datasets/evalset.csv"

column_mapping = {
    "chat_history": "${data.chat_history}",
    "query": "${data.query}",
}

run_cosmos_nosql_topk3 = pf.run(
    flow=flow_path, data=data_path, column_mapping=column_mapping
)
pf.stream(run_cosmos_nosql_topk3)
print(run_cosmos_nosql_topk3)

In [ ]:
pf.get_details(run_cosmos_nosql_topk3)

In [ ]:
ddf = pf.get_details(run_cosmos_nosql_topk3)
# ddf.to_csv("checkoutput.csv")

## Evaluate 

In [ ]:
eval_flow_path = "../../evaluator/eval_aistudio_score/"
data_path = "../../datasets/evalset.csv"

eval_cosmos_nosql_run_topk3 = pf.run(
    flow=eval_flow_path,
    run=run_cosmos_nosql_topk3,
    data=data_path,
    column_mapping={
        "question": "${data.query}",
        "answer": "${data.answer}",
        "reply": "${run.outputs.reply}",
        "context": "${run.outputs.fetched_docs}",
        "log_params": "${run.outputs.log_params}",
    },
    display_name="eval_cosmos_nosql_aistudio_score_topk3",
)
pf.stream(eval_cosmos_nosql_run_topk3)
print(eval_cosmos_nosql_run_topk3)

In [ ]:
pf.get_details(eval_cosmos_nosql_run_topk3)

In [ ]:
pf.get_metrics(eval_cosmos_nosql_run_topk3)

In [ ]:
pf.visualize(eval_cosmos_nosql_run_topk3)

### Multiple runs 
Using config.yaml file

In [ ]:
import yaml
import promptflow

pf = promptflow.PFClient()

with open("./runs_config.yaml", "r") as file:
    runs_config = yaml.safe_load(file)

# Access the contents of the config.yaml file
print(runs_config)

In [ ]:
# function to update param_config systematically
import os


def update_param_config(flow_path, config):
    base_dir = os.path.dirname(flow_path)
    param_config_path = os.path.join(base_dir, "param_config.yaml")

    with open(param_config_path, "r") as file:
        param_dict = yaml.safe_load(file)

    param_dict.update(config)

    with open(param_config_path, "w") as file:
        yaml.dump(param_dict, file)

In [ ]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def run_dags(
    flow_path="../../rag-cosmos-nosql/flow.dag.yaml",
    data_path="../../datasets/evalset.csv",
    eval_flow_path="../../evaluator/eval_aistudio_score/",
    config=None,
):
    column_mapping = {
        "chat_history": "${data.chat_history}",
        "query": "${data.query}",
    }
    update_param_config(flow_path, config)

    copilot_run_from_config = pf.run(
        flow=flow_path,
        data=data_path,
        column_mapping=column_mapping,
        display_name="rag_nosql_run_from_config_" + run_time,
    )

    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
        flow=eval_flow_path,
        run=copilot_run_from_config,
        data=data_path,
        column_mapping={
            "question": "${data.query}",
            "answer": "${data.answer}",
            "reply": "${run.outputs.reply}",
            "context": "${run.outputs.fetched_docs}",
            "log_params": "${run.outputs.log_params}",
        },
        display_name="eval_run_nosql_aistudio_scores_from_config_" + run_time,
    )
    return copilot_run_from_config, eval_run_from_config

In [ ]:
i = 0
run_logs = {}
for run in runs_config:
    i += 1
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(config=runs_config[run])
    run_logs[i] = eval_run_from_config

In [ ]:
for run_id in run_logs:
    print(run_id)
    pf.visualize(run_logs[run_id])